In [13]:
# 引入包
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import time
import bs4
import json
from datetime import datetime

In [14]:
# 随机产生user_agent，避免被禁
def get_agent_pc():
    user_agent_pc = [
        # 谷歌
        'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.html.2171.71 Safari/537.36',
        'Mozilla/5.0.html (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.html.1271.64 Safari/537.11',
        'Mozilla/5.0.html (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.html.648.133 Safari/534.16',
        # 火狐
        'Mozilla/5.0.html (Windows NT 6.1; WOW64; rv:34.0.html) Gecko/20100101 Firefox/34.0.html',
        'Mozilla/5.0.html (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10',
        # opera
        'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.html.2171.95 Safari/537.36 OPR/26.0.html.1656.60',
        # qq浏览器
        'Mozilla/5.0.html (compatible; MSIE 9.0.html; Windows NT 6.1; WOW64; Trident/5.0.html; SLCC2; .NET CLR 2.0.html.50727; .NET CLR 3.5.30729; .NET CLR 3.0.html.30729; Media Center PC 6.0.html; .NET4.0C; .NET4.0E; QQBrowser/7.0.html.3698.400)',
        # 搜狗浏览器
        'Mozilla/5.0.html (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.html.963.84 Safari/535.11 SE 2.X MetaSr 1.0.html',
        # 360浏览器
        'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.html.1599.101 Safari/537.36',
        'Mozilla/5.0.html (Windows NT 6.1; WOW64; Trident/7.0.html; rv:11.0.html) like Gecko',
        # uc浏览器
        'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.html.2125.122 UBrowser/4.0.html.3214.0.html Safari/537.36',
    ]
    return random.choice(user_agent_pc)

In [15]:
# 请求的cookie
cookies = {
    'qgqp_b_id': 'd140d591678662b7c1b9fb74c1ad64ab',
    'HAList': 'ty-1-000001-%u4E0A%u8BC1%u6307%u6570%2Cty-1-601318-%u4E2D%u56FD%u5E73%u5B89%2Cty-1-000002-%uFF21%u80A1%u6307%u6570%2Cty-1-600519-%u8D35%u5DDE%u8305%u53F0',
    'st_si': '52449960593274',
    'st_asi': 'delete',
    'st_pvi': '39951595300255',
    'st_sp': '2023-03-06%2009%3A49%3A34',
    'st_inirUrl': 'https%3A%2F%2Fcn.bing.com%2F',
    'st_sn': '11',
    'st_psi': '20230406125155117-113200313003-9059891318',
}
# 请求头
headers = {
    'Accept': '*/*',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Referer': 'http://quote.eastmoney.com/',
    'User-Agent': get_agent_pc(),
}
# 参数设置
jq = re.sub('\D','','1.12.4'+str(random.random()))
new_ts = time.time()
tm = int(new_ts*1000)
jQuery = 'jQuery{}_{}'.format(jq,tm)
params={'cb':jQuery,
        "pn":1,
        "pz":50,
        "po":1,
        "np":1,
        "ut":"bd1d9ddb04089700cf9c27f6f7426281",
        "fltt":2,
        "invt":2,
        "wbp2u":"|0|0|0|web",
        "fid":"",
        "fs":"b:MK0010",
        "fields":"f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f26,f22,f11,f62,f128,f136,f115,f152",
        '_':'{}'.format(tm)
        }

In [16]:
# 设置存储的dataframe
stock_df = pd.DataFrame(columns=["时间",'代码','名称',"最新价","涨跌幅",'涨跌额',
                                        "成交量","成交额","昨收",'今开','最高','最低'])

In [17]:
# 爬取数据的链接
url = 'http://43.push2.eastmoney.com/api/qt/clist/get'
# 发送请求
response = requests.get(
    url=url,
    cookies=cookies,
    headers=headers,
    params=params
)

# 返回请求状态
print(response.status_code)
if response.status_code==200:
    # 如果成功
    response.encoding = "utf-8"
    text = response.text
    #解析返回的内容
    html = text.lstrip(jQuery+'(')
    html = html.rstrip(');')
    #转换为json数据
    data_json = json.loads(html)
    data = data_json['data']
    # 获取我们要的数据
    rows = data['diff']
    print(data)
    # ["时间",'代码','名称',"最新价","涨跌幅",'涨跌额',"成交量","成交额","昨收",'今开','最高','最低']
    for row in rows:
        # 将dict结构转化为dataframe
        code = row['f12']
        name = row['f14']
        new = row['f2']
        dt = datetime.fromtimestamp(new_ts)
        new_time = dt.strftime("%Y-%m-%d %H:%M:%S")
        price = row['f4']
        price_diff = row['f3']
        turnover = row['f5']
        turnover_num = row['f6']
        yest =row['f18']
        today = row['f17']
        hight = row['f15']
        low = row['f16']
        stock_df.loc[len(stock_df)]=[new_time,code,name,new,price,price_diff,turnover,turnover_num,yest,today,hight,low]


200
{'total': 29, 'diff': [{'f1': 2, 'f2': 3312.63, 'f3': 0.0, 'f4': 0.07, 'f5': 327155424, 'f6': 506500167165.2, 'f7': 0.56, 'f8': 0.76, 'f9': '-', 'f10': 1.01, 'f11': 0.02, 'f12': '000001', 'f13': 1, 'f14': '上证指数', 'f15': 3314.53, 'f16': 3296.06, 'f17': 3302.75, 'f18': 3312.56, 'f20': 50266679099289, 'f21': 43573234197680, 'f22': 0.02, 'f23': '-', 'f24': 6.05, 'f25': 7.23, 'f26': '-', 'f62': -4246949888.0, 'f115': '-', 'f128': '-', 'f140': '-', 'f141': '-', 'f136': '-', 'f152': 2}, {'f1': 2, 'f2': 11866.31, 'f3': 0.06, 'f4': 6.83, 'f5': 454537865, 'f6': 686785473371.8846, 'f7': 0.81, 'f8': 2.05, 'f9': '-', 'f10': 1.03, 'f11': -0.01, 'f12': '399001', 'f13': 0, 'f14': '深证成指', 'f15': 11874.8, 'f16': 11778.8, 'f17': 11787.35, 'f18': 11859.48, 'f20': 35372892133903, 'f21': 28999665089958, 'f22': -0.01, 'f23': '-', 'f24': 6.95, 'f25': 7.72, 'f26': '-', 'f62': -19501633536.0, 'f115': '-', 'f128': '-', 'f140': '-', 'f141': '-', 'f136': '-', 'f152': 2}, {'f1': 2, 'f2': 940.16, 'f3': -0.89, 'f

In [18]:
# 输出前5行
pd.set_option('display.width', 100)
stock_df.head(10)

,时间,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,昨收,今开,最高,最低
0,2023-04-06 19:22:03,000001,上证指数,3312.63,0.07,0.00,327155424,5.065e+11,3312.56,3302.75,3314.53,3296.06
1,2023-04-06 19:22:03,399001,深证成指,11866.31,6.83,0.06,454537865,6.868e+11,11859.48,11787.35,11874.80,11778.80
2,2023-04-06 19:22:03,899050,北证50,940.16,-8.46,-0.89,617194,5.825e+08,948.62,947.06,947.90,937.70
3,2023-04-06 19:22:03,000300,沪深300,4096.64,-6.47,-0.16,150703374,3.290e+11,4103.11,4084.13,4098.83,4078.40
4,2023-04-06 19:22:03,399005,中小100,7857.46,5.71,0.07,36973525,9.669e+10,7851.75,7791.25,7867.25,7781.81
5,2023-04-06 19:22:03,399006,创业板指,2426.57,4.92,0.20,164546058,3.172e+11,2421.65,2404.26,2428.80,2402.30
6,2023-04-06 19:22:03,000010,上证180,8610.58,-15.11,-0.18,100081035,1.810e+11,8625.69,8594.55,8624.22,8579.49
7,2023-04-06 19:22:03,000016,上证50,2666.45,-9.47,-0.35,29605541,5.847e+10,2675.92,2668.13,2672.72,2659.91
8,2023-04-06 19:22:03,000009,上证380,6016.52,33.04,0.55,57852051,1.009e+11,5983.48,5980.05,6021.84,5974.94
9,2023-04-06 19:22:03,000132,上证100,6498.92,40.25,0.62,9124558,2.062e+10,6458.67,6451.69,6502.87,6446.60


In [19]:
# 保存成文件
stock_df.to_csv("沪深重要指数.csv",index=False)